In [3]:
import numpy as np
# import nltk
# nltk.download('punkt')
import pandas as pd
import random
import re
import stanza

In [4]:
# Read in synthetic data
df = pd.read_csv('../../data/data/generated/data_230124-172021.csv')

In [7]:
nlp = stanza.Pipeline(lang='en', processors='tokenize')
df['text_tokenize'] = df['text'].apply(lambda x: nlp(x))

# df['text_split'] = df['text'].apply(lambda x: x.strip())
# df['text_split'] = df['text_split'].apply(lambda x: nltk.word_tokenize(x))

string_cols = ["item1", "item2", ]
df[string_cols] = df[string_cols].astype(str)
df.head()
# df.info()
df['text_tokenize'][2]

2023-02-20 14:51:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-20 14:51:57 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2023-02-20 14:51:57 INFO: Use device: cpu
2023-02-20 14:51:57 INFO: Loading: tokenize
2023-02-20 14:51:57 INFO: Done loading processors!


[
  [
    {
      "id": 1,
      "text": "Plastic",
      "start_char": 1,
      "end_char": 8
    },
    {
      "id": 2,
      "text": "Pollution",
      "start_char": 9,
      "end_char": 18
    },
    {
      "id": 3,
      "text": "Coalition",
      "start_char": 19,
      "end_char": 28
    },
    {
      "id": 4,
      "text": "Australia",
      "start_char": 29,
      "end_char": 38
    },
    {
      "id": 5,
      "text": "(",
      "start_char": 39,
      "end_char": 40
    },
    {
      "id": 6,
      "text": "PPCA",
      "start_char": 40,
      "end_char": 44
    },
    {
      "id": 7,
      "text": ")",
      "start_char": 44,
      "end_char": 45
    },
    {
      "id": 8,
      "text": "is",
      "start_char": 46,
      "end_char": 48
    },
    {
      "id": 9,
      "text": "proud",
      "start_char": 49,
      "end_char": 54
    },
    {
      "id": 10,
      "text": "to",
      "start_char": 55,
      "end_char": 57
    },
    {
      "id": 11,
      "text": "

In [27]:
# for sentence in df['text_tokenize'][2].sentences:
#     print(len(sentence.words))
#     # for i in sentence.words:
#     #     print(i.text)
    

In [23]:
# Convert data into list of words with associated 'B - entity', 'I - entity' or 'O'
# Look at other preprocessing steps in read_datasets function in convert_bn_daffodil

units = ["kilograms", "kilogram", "kgs", "kg", "lb", "lbs", "pounds", "pound"]

def assign_entity_types(row):
    doc = row['text_tokenize']
    new_tags = []
    prev_org_tag = False
    prev_loc_tag = False
    prev_unit_tag = False
    prev_weight_tag = False
    prev_item_tag = False
    for sentence in doc.sentences:
        for i in range(len(sentence.words)):
            # Assign location labels
            if any(sentence.words[i] == word for word in row['location'].split()): #Problem: only searches for specific location that was 
                                            # given in the prompt, not other locations that GPT-3 produces in text
                                            # Solution: implement another check for all international countries?
                if prev_loc_tag:
                    new_tags.append("I-LOC")
                else:
                    new_tags.append("B-LOC")
                    prev_loc_tag = True
            # Assign organization labels
            elif any(sentence.words[i] == word for word in row['organization'].split()):
                if prev_org_tag:                    # consider capitalization - should i lowercase everything when checking orgs?
                    new_tags.append("I-ORG")
                else:
                    new_tags.append("B-ORG")
                    prev_org_tag = True
            # Assign unit labels
            elif any(sentence.words[i] == word for word in units):
                if prev_unit_tag: 
                    new_tags.append("I-UNT")
                else:
                    new_tags.append("B-UNT")
                    prev_unit_tag = True
            # Assign weight labels 
            elif (sentence.words[i] == str(row['weight1']) or (row['weight2'] != None and sentence.words[i] == str(row['weight2']))):
                if prev_weight_tag: 
                    new_tags.append("I-WEI")
                else:
                    new_tags.append("B-WEI")
                    prev_weight_tag = True
            # Assign item labels
            elif (any(sentence.words[i] == word for word in row['item1'].split()) or 
                                    (row['item2'] != None and any(sentence.words[i] == word for word in row['item2'].split()))):
                if prev_item_tag: 
                    new_tags.append("I-ITM")
                else:
                    new_tags.append("B-ITM")
                    prev_item_tag = True
            # Open question: How to assign dates? Need to capture all possible date formats?
            else:
                new_tags.append("O")
                prev_org_tag = False
                prev_loc_tag = False
                prev_unit_tag = False
                prev_weight_tag = False
                prev_item_tag = False
    return list(zip(sentence.words, new_tags))

df['tagged_entities'] = df.apply(lambda row : assign_entity_types(row), axis =1)


In [26]:
# Review newly assigned non-"O" tags
SAMPLE_NO = 2
for i in df.iloc[SAMPLE_NO]['tagged_entities']:
    if i[1] != "O":
        print(i[0], i[1])

print(df['tagged_entities'][SAMPLE_NO])
df['text'][2]

[({
  "id": 1,
  "text": "\"",
  "start_char": 1046,
  "end_char": 1047
}, 'O'), ({
  "id": 2,
  "text": "We",
  "start_char": 1047,
  "end_char": 1049
}, 'O'), ({
  "id": 3,
  "text": "are",
  "start_char": 1050,
  "end_char": 1053
}, 'O'), ({
  "id": 4,
  "text": "calling",
  "start_char": 1054,
  "end_char": 1061
}, 'O'), ({
  "id": 5,
  "text": "on",
  "start_char": 1062,
  "end_char": 1064
}, 'O'), ({
  "id": 6,
  "text": "all",
  "start_char": 1065,
  "end_char": 1068
}, 'O'), ({
  "id": 7,
  "text": "Australians",
  "start_char": 1069,
  "end_char": 1080
}, 'O'), ({
  "id": 8,
  "text": "to",
  "start_char": 1081,
  "end_char": 1083
}, 'O'), ({
  "id": 9,
  "text": "join",
  "start_char": 1084,
  "end_char": 1088
}, 'O'), ({
  "id": 10,
  "text": "us",
  "start_char": 1089,
  "end_char": 1091
}, 'O'), ({
  "id": 11,
  "text": "in",
  "start_char": 1092,
  "end_char": 1094
}, 'O'), ({
  "id": 12,
  "text": "this",
  "start_char": 1095,
  "end_char": 1099
}, 'O'), ({
  "id": 13,
 

'\nPlastic Pollution Coalition Australia (PPCA) is proud to announce the success of its latest beach cleanup on Norfolk Island. On May 25th, 2021, PPCA and its volunteers collected a whopping 386 kilograms of glass bottles from the island\'s coastline.\n\nThis is a remarkable achievement for the organization, which has made it its mission to rid Australia\'s oceans and beaches of plastic pollution.\n\n"It is extremely gratifying for us to see the tangible results of our work in beach cleanups like this one," said PPCA CEO John Smith. "Our volunteers worked hard and achieved a great result, and we are thrilled that we were able to make such a big difference in the local environment."\n\nThe glass bottles collected during the cleanup will be recycled and reused. This not only helps to reduce the amount of plastic waste in the ocean, but also supports the local economy by creating jobs and generating revenue.\n\n"We are committed to continuing our efforts to make Australia\'s beaches and 

In [ ]:
output_file_name = 'test.tsv'
# df['tagged_entities'].to_csv(output_file_name, sep = "\t")


In [ ]:
# Compile all sentences into a single list of lists (sentences) of word-pairs (word, NER tag)

# Open question: should I drop punctuation from training sentences?

import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize')
doc = nlp('This is a test sentence for stanza. This is another sentence.')
for i, sentence in enumerate(doc.sentences):
    print(f'====== Sentence {i+1} tokens =======')
    print(*[f'id: {token.id}\ttext: {token.text}' for token in sentence.tokens], sep='\n')



In [ ]:
# Divide data into datasets = (train_sentences, dev_sentences, test_sentences)

random.seed(1234)
random.shuffle(train_sentences)
split_len = len(train_sentences) * 9 // 10
dev_sentences = train_sentences[split_len:]
train_sentences = train_sentences[:split_len]

In [ ]:
# Convert to JSON file needed by Stanza model
# There is a conversion script called several times in prepare_ner_dataset.py which converts IOB format to our internal NER format:
# import stanza.utils.datasets.ner.prepare_ner_file as prepare_ner_file

# prepare_ner_file.process_dataset(input_iob, output_json)

In [ ]:
# Or call write_dataset function from stanza.utils.datasets.ner.utils (docstring below)?
def write_dataset(datasets, output_dir, short_name, suffix="bio"):
    """
    write all three pieces of a dataset to output_dir

    datasets should be 3 lists: train, dev, test
    each list should be a list of sentences
    each sentence is a list of pairs: word, tag

    after writing to .bio files, the files will be converted to .json
    """
    pass